<b> Word Similarity </b> <br>
Your next step is to evaluate the three kinds of embeddings in the word similarity
task. The goal of this task is to compute similarity of two words and
evaluate the model-produced similarity against human judgements. Download
two commonly-used word similarity datasets:<br>
• SimLex: https://www.cl.cam.ac.uk/˜fh295/simlex.html<br>
• MEN: https://staff.fnwi.uva.nl/e.bruni/MEN<br>
Compute cosine similarity between the words using the three models. Evaluate
the model-produced similarities against human judgements in terms of
Pearson and Spearman correlation coefficients. <br>
Compare the performance of
the three models on this task. <br>
Analyze the data qualitatively and report what
are the differences in the kind of similarity captured by the three models.<br> We
are interested to see both quantitative results and qualitative analysis in your
report.

In [3]:
def read_in_SimLex():
    # Read in the SimLex dataset
    SimLex_file = open('SimLex-999/SimLex-999.txt','r')
    SimLex_set = {}
    next(SimLex_file)
    for line in SimLex_file:
        pair_data = [x.strip() for x in line.split('\t')]
        word1 = pair_data[0]
        word2 = pair_data[1]
        similarity_score = pair_data[4]
        SimLex_set[(word1, word2)] = {'human':similarity_score}
    return SimLex_set


In [4]:
# Read in the MEN dataset
def read_in_MENS():
    MEN_file = open('MEN/MEN_dataset_natural_form_full','r')
    MEN_set = {}
    for line in MEN_file:
        pair_data = [x.strip() for x in line.split(' ')]
        word1 = pair_data[0]
        word2 = pair_data[1]
        similarity_score = pair_data[2]
        MEN_set[(word1, word2)] = {'human':similarity_score}
    return MEN_set

In [5]:
# In order to reduce the size of the word embedding files,
#  I will delete all non-occuring words from the relevant model and
# store the word embeddings in a defaultdict for quick access.
def get_occuring_words(MEN_set, SimLex_set):
    occuring_words = []
    for pair in list(MEN_set.keys())+list(SimLex_set.keys()):
        word1 = pair[0]
        word2 = pair[1]
        if word1 not in occuring_words:
            occuring_words.append(word1)
        if word2 not in occuring_words:
            occuring_words.append(word2)
    return occuring_words

In [6]:
# Read in the word vectors for a given word model
import os
import time
from collections import defaultdict

# switch between 'bow5.words', 'bow2.words' and 'deps.words'
def get_embeddings_similarity(filename, occuring_words):
    # Returns a dictionary containing words as keys and word vectors
    #  as values
    if filename+'_small' in os.listdir():
        f = open(filename+'_small')
        using_small = True
    else:
        f = open(filename,'r')
        d = open(filename+'_small','w')
    word_embeddings = defaultdict(list)
    i =0
    s = time.time()
    for line in f:
        entry = [x.strip() for x in line.split(' ')]
        word = entry[0]
        vector = entry[1:]
        if word in occuring_words:
            word_embeddings[word] = [np.float(x) for x in vector]
            if not using_small:
                d.write(line)
    if not using_small:
        d.close()
    f.close()
    e = time.time()
    print('It took {} seconds to read in dataset {}'.format(e-s,filename))
    return word_embeddings



In [77]:
import numpy as np
import math
def cosine_similarity(v1, v2,n1, n2):
    # Returns a value between -1 and 1, 1 meaning exactly same
    #  and -1 meaning exactly opposite.
    assert len(v1) == len(v2)
    numerator = sum([v1[i]*v2[i] for i in range(len(v1))])
    denominator = np.sqrt(sum([x**2 for x in v1])) \
                * np.sqrt(sum([x**2 for x in v2]))
    if denominator == 0:
        print('division by zero with words:')
        print(n1)
        print(n2)
    return (numerator/denominator)

In [8]:

def add_cosine_similarity(dataset, word_embeddings):
    # Computes the cosine similarity per word pair and adds
    #  it to the pair dictionary entry.
    for pair in list(dataset.keys()):
        v1, v2 = word_embeddings[pair[0]], word_embeddings[pair[1]]
        if len(v1) != len(v2):
            if len(v1) == 0 or len(v2) == 0:
                print('Removing {} and {} from dataset'.format(pair[0],pair[1]))
                print('No word embedding found for one of them')
                del dataset[pair]
            else:
                print("Something weird happened with:")
                print(pair[0], pair[1])
        else:
            cos_sim = cosine_similarity(v1,v2,pair[0],pair[1])
            dataset[pair]['cos_sim'] = cos_sim
    return dataset

def get_just_data(dataset):
    # Returns the human judgements and cosine similarities 
    # as a nx2 numpy array for easy data anaylsis
    n = len(dataset)
    result = np.zeros((2,n))
    for i,pair in enumerate(list(dataset.keys())):
        result[0,i] = dataset[pair]['human']
        result[1,i] = dataset[pair]['cos_sim']
    return result



In [9]:
def get_similarity_measures(filename):
    s = time.time()
    MEN_set = read_in_MENS()
    SimLex_set = read_in_SimLex()
    occuring_words = get_occuring_words(MEN_set, SimLex_set)
    word_embeddings = get_embeddings_similarity(filename,occuring_words)
    MEN_set = add_cosine_similarity(MEN_set, word_embeddings)
    SimLex_set = add_cosine_similarity(SimLex_set, word_embeddings)
    MEN_data = get_just_data(MEN_set)
    SimLex_data = get_just_data(SimLex_set)
    e = time.time()
    print('Getting similarity measures on {} took {} seconds'.format(filename,(e-s)))
    return MEN_data, SimLex_data

In [14]:
# Read in the three datasets
MEN_k2, Sim_k2 = get_similarity_measures('bow2.words')
MEN_k5, Sim_k5 = get_similarity_measures('bow5.words')
MEN_deps, Sim_deps = get_similarity_measures('deps.words')


NameError: global name 'get_occuring_words' is not defined

In [ ]:
import scipy.stats as ss

def precision(data,k):
    n = len(data[0,:])
    d = data[:,data[0,:]]
    return None
data = MEN_deps
d = data[:,data[0,:].argsort()]
rg_x = ss.rankdata(d[0,:],'dense')
rg_y = ss.rankdata(d[1,:],'dense')

n = len(data[0,:])
for i in range(n):
    print(d[:,i],rg_x[i],rg_y[i])
    


In [ ]:
import matplotlib.pyplot as plt
import scipy.stats as ss

def calc_Pearson_correlation(data):
    # data[0,:] are the human observations
    # data[1,:] are the computed cosine similarities
    assert data.shape[0] == 2, 'Data must have two measures per variable.'
    cov_matrix = np.cov(data)
    assert cov_matrix[1,0] == cov_matrix[0,1],'These should be same...'
    cov_hum_cos = cov_matrix[1,0]
    std_hum = np.std(data[0,:])
    std_cos = np.std(data[1,:])
    rho = cov_hum_cos/(std_hum*std_cos)
    return rho

def calc_Spearman_correlation(data):
    # Source https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient
    n = len(data[0,:])
    x = data[0,:]
    y = data[1,:]
    rg_x = ss.rankdata(x)
    rg_y = ss.rankdata(y)
    rg_data = np.array([rg_x,rg_y])
    cov_matrix = np.cov(rg_data)
    assert cov_matrix[1,0] == cov_matrix[0,1],'These should be same...'
    cov_rg_xy = cov_matrix[1,0]
    std_rg_x = np.std(rg_x)
    std_rg_y = np.std(rg_y)
    rho_rg = cov_rg_xy/(std_rg_x*std_rg_y)
    return rho_rg

def pearson_bootstrap_test(data, dataset_name, model_name):
    n = len(data[0,:])
    no_samples = 10000
    sample_size = int(np.floor(n/20))
    rho_full = calc_Pearson_correlation(data)
    rho_samples = []
    for i in range(no_samples):
        rand_idx = np.random.choice(range(n),n,replace=True,)
        sample_data = data[:,rand_idx]
        rho_sample = calc_Pearson_correlation(sample_data)
        rho_samples.append(rho_sample)
    rho_samples.sort()
    rho_lower = rho_samples[int(no_samples*.025)]
    rho_upper = rho_samples[int(no_samples*.975)]
    plt.title('Pearson Correlation Coefficient Boostrap test \n Dataset: {} \n Model: {}'.format(dataset_name, model_name))
    counts,b,p = plt.hist(rho_samples,bins=100)
    label_height = max(counts)
    plt.xlabel(r'$\rho$')
    plt.ylabel('Count')
    plt.axvline(x=rho_full,c='red')
    plt.text(rho_full,label_height,r'$\rho_{pop}$'+": {0:.{1}f}".format(rho_full,3))
    plt.axvline(x=rho_lower,c='red')
    plt.text(rho_lower,label_height,r'$\rho_{lo}$'+": {0:.{1}f}".format(rho_lower,3))
    plt.axvline(x=rho_upper,c='red')
    plt.text(rho_upper,label_height,r'$\rho_{up}$'+": {0:.{1}f}".format(rho_upper,3))
    plt.show()
    return rho_samples


#plt.scatter(Sim_k5[1,:],Sim_k5[0,:])


In [ ]:
# Pearson correlation bootstrap tests

#pearson_bootstrap_test(MEN_k2,'MEN','BOW with k=2')
#pearson_bootstrap_test(MEN_k5,'MEN','BOW with k=5')
#pearson_bootstrap_test(MEN_deps,'MEN','Dependency based')
#pearson_bootstrap_test(Sim_k2,'SimLex','BOW with k=2')
#pearson_bootstrap_test(Sim_k5,'SimLex','BOW with k=5')
#pearson_bootstrap_test(Sim_deps,'SimLex','Dependency based')




In [ ]:
print(calc_Spearman_correlation(MEN_k2))
print(calc_Spearman_correlation(MEN_k5))
print(calc_Spearman_correlation(MEN_deps))
print(calc_Spearman_correlation(Sim_k2))
print(calc_Spearman_correlation(Sim_k5))
print(calc_Spearman_correlation(Sim_deps))

In [10]:
# switch between 'bow5.words', 'bow2.words' and 'deps.words'
def get_embeddings(filename):
    # Returns a dictionary containing words as keys and word vectors
    #  as values
    
    f = open(filename,'r')
    word_embeddings = defaultdict(list)
    s = time.time()
    for line in f:
        entry = [x.strip() for x in line.split(' ')]
        word = entry[0]
        vector = entry[1:]
        
        word_embeddings[word] = [np.float(x) for x in vector]
    f.close()
    e = time.time()
    print('It took {} seconds to read in dataset {}'.format(e-s,filename))
    return word_embeddings

bow5_embeddings = get_embeddings('bow5.words')
# bow2_embeddings = get_embeddings('bow2.words')
# deps_embeddings = get_embeddings('deps.words')

It took 129.414999962 seconds to read in dataset bow5.words


In [ ]:
def compare(b1_estimate, embeddings):
    min_cos = 1000000
    best_word = None
    for word in list(embeddings.keys()):
        word_vector = embeddings[word]
        cos_sim = cosine_similarity(b1_estimate, word_vector, None, None)
        if cos_sim < min_cos:
            min_cos = cos_sim
            best_word = word
        
    return best_word

In [11]:
import pandas as pd
i = 0
n = len(list(bow5_embeddings.keys()))
X = np.zeros((n,300))
y = []
for i,word in enumerate(list(bow5_embeddings.keys())):
    word_vector = np.array(bow5_embeddings[word])
    X[i,:] = word_vector
    y.append(word)
df = pd.DataFrame(X)
df['label'] = y
df['label'] = df['label'].apply(lambda i: str(i))
print(df[0])

0        -0.013674
1        -0.050265
2        -0.057644
3         0.059332
4         0.021111
5        -0.021032
6        -0.113992
7         0.045993
8        -0.022300
9        -0.018712
10       -0.050854
11       -0.027307
12        0.033520
13        0.009165
14       -0.066100
15        0.027763
16       -0.149122
17        0.030789
18       -0.045150
19       -0.169552
20       -0.021733
21       -0.087136
22        0.000037
23        0.021034
24       -0.053521
25        0.040087
26       -0.073763
27       -0.022306
28        0.011079
29       -0.009930
            ...   
183840   -0.003072
183841   -0.006145
183842    0.005234
183843   -0.001776
183844   -0.041788
183845   -0.095522
183846   -0.038766
183847   -0.157794
183848   -0.023214
183849   -0.057377
183850   -0.007204
183851   -0.027025
183852   -0.035300
183853   -0.028800
183854    0.017544
183855   -0.049424
183856    0.020135
183857   -0.036307
183858   -0.014736
183859   -0.061745
183860    0.042964
183861   -0.

In [46]:
rndperm = np.random.permutation(df.shape[0]) # Random indices
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
pca_result = pca.fit_transform(df[range(n)].values)


183870


## Word Analogy Task

In [21]:
import heapq
from heapq import heappush, heappushpop
from scipy.spatial.distance import cosine


In [36]:
# def read_google_file():
#     test = []
#     with open('questions-words.txt') as f:
#         for line in f:
#             analogy = line.strip().lower().split()
#             test.append(analogy)
#     return test 

In [96]:
test = []
with open('questions-words.txt') as f:
    for line in f:
        analogy = line.strip().lower().split()
        if(analogy[0] is not ":"):
            test.append(analogy)

In [108]:
test_array = np.array([np.array(i) for i in test])

In [129]:
np.asarray(test_array).shape

(19544L, 4L)

In [152]:
a0 = [column[0] for column in test_array]
a1 = [column[1] for column in test_array]
b0 = [column[2] for column in test_array]
b1 = [column[3] for column in test_array]


In [153]:
def offset_vector(a0, a1, b0, b1):
    return cosine(b1, a1 - a0 + b0)
#     bi_estimate = (a1 - a0) + b0
#     return bi_estimate
    

In [154]:
def get_nearest(vectors, distance, N = 10):
    nearest = []
    for word,u in vectors.items():
        nearness = -distance(u)
        if len(nearest) < N:
            heapq.heappush(nearest, (nearness, word))
        else:
            heapq.heappushpop(nearest, (nearness, word))
    return sorted(nearest, reverse=True)

In [155]:
def print_nearest(vectors, word):
    v = vectors[word]
    print(word, np.linalg.norm(v))
    for nearness, word in get_nearest(vectors, lambda u: cosine(u, v)):
        print(word, nearness)

In [156]:
def get_analogues(vectors, a0, a1, b0, b1, distance):
    return get_nearest(vectors, lambda b1: distance(a0, a1, b0, b1))


In [158]:
def print_analogues(vectors, a0, a1, b1, distance=offset_vector):
    print(a0, a1, b0)
    for nearness, word in get_analogues(vectors, vectors[a0], vectors[a1], vectors[b0],
                                    distance):
        print(word, nearness)
        

In [173]:
def main():
    vectors = bow5_embeddings.keys()
    print_analogues(vectors, a0, a1, b0, distance=offset_vector)

In [174]:
if __name__ == '__main__':
    main()

(['athens', 'athens', 'athens', 'athens', 'athens', 'athens', 'athens', 'athens', 'athens', 'athens', 'athens', 'athens', 'athens', 'athens', 'athens', 'athens', 'athens', 'athens', 'athens', 'athens', 'athens', 'athens', 'baghdad', 'baghdad', 'baghdad', 'baghdad', 'baghdad', 'baghdad', 'baghdad', 'baghdad', 'baghdad', 'baghdad', 'baghdad', 'baghdad', 'baghdad', 'baghdad', 'baghdad', 'baghdad', 'baghdad', 'baghdad', 'baghdad', 'baghdad', 'baghdad', 'baghdad', 'bangkok', 'bangkok', 'bangkok', 'bangkok', 'bangkok', 'bangkok', 'bangkok', 'bangkok', 'bangkok', 'bangkok', 'bangkok', 'bangkok', 'bangkok', 'bangkok', 'bangkok', 'bangkok', 'bangkok', 'bangkok', 'bangkok', 'bangkok', 'bangkok', 'bangkok', 'beijing', 'beijing', 'beijing', 'beijing', 'beijing', 'beijing', 'beijing', 'beijing', 'beijing', 'beijing', 'beijing', 'beijing', 'beijing', 'beijing', 'beijing', 'beijing', 'beijing', 'beijing', 'beijing', 'beijing', 'beijing', 'beijing', 'berlin', 'berlin', 'berlin', 'berlin', 'berlin', 'b

TypeError: list indices must be integers, not list